In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score, confusion_matrix, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# option d'affichage des résultats
#pd.set_option("display.max_row",130)
pd.set_option("display.max_columns",100)

In [4]:
# les noms des variables

colonnes =(["IDENT", "TypeHabi", "GeoFaz_Scl", "GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Anciennete",
            "Logement_prix_m2", "Mediane_RFR", "Sexe", "Prenom_Phase_scl_H", "Prenom_Phase_scl_F",
            "Prenom_Phase_cl_H", "Prenom_Phase_cl_F", "Age_H", "Age_F", "Affinite_1", "Affinite_2",
            "Prenom_effectif", "Potentiel_prenom", "Onoma-Phase_Scl", "Onoma-Phase_CL", "Top_mon"])


In [20]:

colonnes_montant = ["DEDUPLI", "IDENT", "SommeDeMONTANT_NUM", "Flag_Top_Montant", "NovosScore_Vingtile"]
colonnes_montant = colonnes_montant[:-1]

data_montant_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/DONS_AA_N_TODAY_1ers gestes_REF_MD.txt", sep=";", header = 0, low_memory=False, dtype=str) #, dtype=str )#, dtype='Int64')
data_montant_orig =data_montant_orig.iloc[:,1:]  

data_montant_orig.iloc[:,2]=data_montant_orig.iloc[:,2].str.replace(",",".")

data_montant_orig.columns =colonnes_montant
data_montant_orig["SommeDeMONTANT_NUM"]=data_montant_orig["SommeDeMONTANT_NUM"].astype(float)


In [5]:
data_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/sep_AAPOT21.txt", delim_whitespace="\t", header = None, names=colonnes, low_memory=False, dtype=str )#, dtype='Int64')
data = data_orig.copy()
data

,IDENT,TypeHabi,GeoFaz_Scl,GeoFaz_cl,ImmoFaz_Scl,ImmoFaz_cl,Anciennete,Logement_prix_m2,Mediane_RFR,Sexe,Prenom_Phase_scl_H,Prenom_Phase_scl_F,Prenom_Phase_cl_H,Prenom_Phase_cl_F,Age_H,Age_F,Affinite_1,Affinite_2,Prenom_effectif,Potentiel_prenom,Onoma-Phase_Scl,Onoma-Phase_CL,Top_mon
0,0010000067,01,72,07,83,08,06,17,18,05,44,81,04,08,06,12,02,03,04,12,31,03,1
1,0010000070,01,72,07,84,08,05,18,20,05,33,44,03,04,08,08,05,09,05,20,36,03,2
2,0010000090,03,64,06,75,07,05,13,08,05,71,67,07,06,12,11,10,08,04,02,22,02,2
3,0010000124,03,43,04,82,08,01,15,17,05,51,81,05,08,10,12,10,14,05,15,61,06,2
4,0010000147,02,25,02,32,03,07,07,15,05,71,81,07,08,12,12,21,21,06,21,61,06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152505,0010382515,02,37,03,22,02,07,03,13,05,71,66,07,06,12,11,12,16,04,07,12,01,2
152506,0010382517,02,31,03,82,08,06,16,20,05,71,64,07,06,12,09,19,19,04,13,13,01,1
152507,0010382520,01,75,07,24,02,07,03,01,05,66,81,06,08,11,12,09,06,04,12,22,02,1
152508,0010382522,02,75,07,43,04,06,05,04,05,71,72,07,07,12,08,15,15,05,13,37,03,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152510 entries, 0 to 152509
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   IDENT               152510 non-null  object
 1   TypeHabi            152510 non-null  object
 2   GeoFaz_Scl          152510 non-null  object
 3   GeoFaz_cl           152510 non-null  object
 4   ImmoFaz_Scl         152510 non-null  object
 5   ImmoFaz_cl          152510 non-null  object
 6   Anciennete          152510 non-null  object
 7   Logement_prix_m2    152510 non-null  object
 8   Mediane_RFR         152510 non-null  object
 9   Sexe                152510 non-null  object
 10  Prenom_Phase_scl_H  152510 non-null  object
 11  Prenom_Phase_scl_F  152510 non-null  object
 12  Prenom_Phase_cl_H   152510 non-null  object
 13  Prenom_Phase_cl_F   152510 non-null  object
 14  Age_H               152510 non-null  object
 15  Age_F               152510 non-null  object
 16  Af

In [7]:
#nombre de modalités de chaque variables (dans un dataframe)
def nombre_mod(data2):
    #création d'un dataframe vide
    dff= pd.DataFrame(columns=["VARIABLE","NOMBRE_MOD", "NOMBRE_MOD_SANS_NA"])
    for i in range(data2.shape[1]): # sans compter les valeurs manquantes
        taille =len(data2.iloc[:,i].value_counts(dropna=False))
        taille_1 = len(data2.iloc[:,i].value_counts(dropna=True) )
        dff=dff.append({"VARIABLE": colonnes[i],"NOMBRE_MOD":taille, "NOMBRE_MOD_SANS_NA":taille_1 }, ignore_index=True)
    return dff

# sélection des corrélations les plus importantes
def variables_corr(tableau_corr, seuil):
    colonnes = tableau_corr.columns
    val_temp = 0 
    variables_corr_pos=[] ; variables_corr_pos_val=[]
    variables_corr_neg=[] ; variables_corr_neg_val=[]
    for i in range(tableau_corr.shape[0]):  
        for j in range(tableau_corr.shape[0]):
            if i<j: #i!=j:
                if (tableau_corr.iloc[i,j] > seuil):
                    variables_corr_pos.append(colonnes[j])
                    variables_corr_pos_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

                if (tableau_corr.iloc[i,j] <-seuil):
                    variables_corr_neg.append(colonnes[j])
                    variables_corr_neg_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

        if (len(variables_corr_pos) > 0) | (len(variables_corr_neg) >0) :
            print("variable : ", colonnes[i])
            if len(variables_corr_pos) > 0:    
                print("corr pos", variables_corr_pos)
                print("corr pos", variables_corr_pos_val)
                val_temp += 1

            if len(variables_corr_neg) >0:  
                print("corr neg", variables_corr_neg)
                print("corr neg", variables_corr_neg_val)
                val_temp += 1
            print("")
        variables_corr_pos = variables_corr_pos*0
        variables_corr_neg = variables_corr_pos*0

        variables_corr_pos_val = variables_corr_pos_val*0
        variables_corr_neg_val = variables_corr_pos_val*0
    if val_temp == 0:
        print("IL N'Y A PAS DE CORRELATION ENTRE LES VARIABLES PAR RAPPORT AU SEUIL PRIS")
        

def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (10,6))
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
    else :
        print("GRAPHIQUE NON DEMANDE")

# Comme il n'y a pas de valeurs manquantes (à la place il y a un nombre), le seul traitement qu'on fait c'est tranformer
# le type de variable (les variables continues deviennent qualitatives)
def data_processing(df):
    """ 
    for i in range(0,len(df.columns)):    
        df[df.columns[i]].fillna(100.0, inplace=True) #On remplace les données manquantes par 100
    for i in range(0,len(df.columns)):
        print(df[df.columns[i]].value_counts(dropna=False))    
    """
    
   #Transformation de toutes les variables en facteurs         
    for i in range(0,len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].astype('category')
    return df.to_numpy() #.as_matrix()


def vingtiles(model, data_orig, affichage, save, path): # data_orig est une variables locale (peut être différente au df d'origine)
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe Top_mon)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # vingtiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    #print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)

    # création de la dernière colonne CLASSE et son remplissage
    index_20_ord = pd.DataFrame(np.unique(df_tous["TRANCHES"])) 
    df_tous["CLASSE"]=df_tous["TRANCHES"].copy()
    ind_val=1
    for kk in range(20):
        df_tous["CLASSE"][df_tous["CLASSE"]== index_20_ord.iloc[kk,0] ] = str(ind_val) 
        ind_val+=1
    print(df_tous)    
    
    # pour la colonne Top_mon : changement de type de variable et remplissage des valeurs (s'il le faut)
    data_temp_t = data_orig.copy()
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t #data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne TOTAL
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

    return  df_tous


def tableau_5_montant(df_tous, data_montant_orig, affichage, save, path):
    # tableau avec les deux variables qui nous intéressent
    ident_et_montant = data_montant_orig[["IDENT","SommeDeMONTANT_NUM"]]

    # jointure
    tableau_resultats_et_montant = pd.merge(df_tous,ident_et_montant)
    #print(tableau_resultats_et_montant) 

    # on élimine toutes les sommes supérieur à 5000 €
    tableau_resultats_et_montant = tableau_resultats_et_montant[tableau_resultats_et_montant["SommeDeMONTANT_NUM"] <= 5000]
    print(tableau_resultats_et_montant) 
    #print(tableau_resultats_et_montant["TRANCHES"].value_counts())
    #print(pd.DataFrame(np.unique(tableau_resultats_et_montant["TRANCHES"])))

    # index
    index_temp = pd.DataFrame(np.unique(tableau_resultats_et_montant["TRANCHES"])) 

    # création du tableau qui contiendra les résultats finaux
    tableau_resultats_finaux = pd.DataFrame(columns=["TRANCHES","Moyen_Somme","TOTAL"],index=range(20))
    tableau_resultats_finaux["TRANCHES"] = index_temp

    # remplissage du tableau final
    for k in range(20):
        tableau_resultats_finaux.iloc[k,1] = tableau_resultats_et_montant[tableau_resultats_et_montant["TRANCHES"]==index_temp.iloc[k,0]]["SommeDeMONTANT_NUM"].mean()
        tableau_resultats_finaux.iloc[k,2] = (tableau_resultats_et_montant["TRANCHES"]==index_temp.iloc[k,0]).sum()
    
    # affichage
    if affichage:        
        print(tableau_resultats_finaux)
    
    # enregistrement sous Excel
    if save:
        tableau_resultats_finaux.to_excel(path)


def vingtiles_ind_sans_target(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    print(df_tous)
    print(df_tous["TRANCHES"].value_counts(ascending = True))

    # GERE L'ORDRE
    patra = np.unique(df_tous.iloc[:,2], return_counts=True)
    df_tranches_sans_target = pd.DataFrame([patra[0],patra[1]]).T
    df_tranches_sans_target.columns=["TRANCHES","COUNTS"]

    # affichage
    if affichage ==True:        
        print(df_tranches_sans_target)
    
    # enregistrement sous Excel
    if save == True:
        df_tranches_sans_target.to_excel(path)
    

# élimination des variables trop corrélées (choix manuel)
def elimination_val_corr(data):
    data = data.drop(columns=[""])
    return data
          


In [8]:
##############################################################################################################
#                                                   PREMIERS TESTS
##############################################################################################################


In [9]:
#################################################################################################
#                       SANS VARIABLES CORR ET ENCODAGE DUMMIES
#################################################################################################


In [10]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F",
                "Onoma-Phase_Scl", "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe Top_mon
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:
#######################################################################################################
#                               REGRESSION LOGISTIQUE
#######################################################################################################

In [ ]:
model_reg_logistic = LogisticRegression(random_state =0, max_iter= 10000, class_weight="balanced" )

param = {'C':[0.01,0.05,0.1,1,2], 'solver': ['liblinear','newton-cg', 'lbfgs'], 'penalty': ["l1","l2","none"] } 

grid_reg_logistic = GridSearchCV(model_reg_logistic, param_grid = param, cv=5, scoring ='recall', n_jobs=2 )
resultat_reg_logistic = grid_reg_logistic.fit(train_X_s, train_Y_s)
print(resultat_reg_logistic.best_params_)


{'C': 0.01, 'penalty': 'none', 'solver': 'lbfgs'}


In [28]:
model_17_cw = LogisticRegression(random_state =0, C=0.01,penalty="none" , solver="lbfgs", max_iter= 10000, class_weight="balanced")

evaluation(model_17_cw, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False )

test set : 
[[ 3495  2610]
 [ 8954 15443]]
              precision    recall  f1-score   support

           1       0.28      0.57      0.38      6105
           2       0.86      0.63      0.73     24397

    accuracy                           0.62     30502
   macro avg       0.57      0.60      0.55     30502
weighted avg       0.74      0.62      0.66     30502

train set : 
[[14247 10087]
 [35791 61883]]
              precision    recall  f1-score   support

           1       0.28      0.59      0.38     24334
           2       0.86      0.63      0.73     97674

    accuracy                           0.62    122008
   macro avg       0.57      0.61      0.56    122008
weighted avg       0.75      0.62      0.66    122008

GRAPHIQUE NON DEMANDE


In [29]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model=model_17_cw, data_orig= data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_logistic_regression.xlsx")

             IDENT     PROBA
55829   0010141500  0.000780
139119  0010354142  0.001011
25069   0010067604  0.001014
119480  0010305739  0.001121
38487   0010102619  0.001134
...            ...       ...
72826   0010187194  0.900752
87312   0010223246  0.915061
25340   0010067908  0.919779
26399   0010069098  0.925294
82689   0010211476  0.925294

[152510 rows x 2 columns]
0.846705    204
0.840589     76
0.890249     60
0.782927     49
0.843653     47
           ... 
0.420471      1
0.333232      1
0.524124      1
0.380102      1
0.477243      1
Name: PROBA, Length: 142315, dtype: int64
             IDENT     PROBA                          TRANCHES CLASSE
55829   0010141500  0.000780  (-0.00022000000000000003, 0.272]      1
139119  0010354142  0.001011  (-0.00022000000000000003, 0.272]      1
25069   0010067604  0.001014  (-0.00022000000000000003, 0.272]      1
119480  0010305739  0.001121  (-0.00022000000000000003, 0.272]      1
38487   0010102619  0.001134  (-0.00022000000000000003, 0

In [30]:
# TABLEAU MONTANT
tableau_5_montant(vingtiles_tableau, data_montant_orig, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/montant_logistic_regression.xlsx")

             IDENT     PROBA                          TRANCHES CLASSE  \
0       0010141500  0.000780  (-0.00022000000000000003, 0.272]      1   
1       0010354142  0.001011  (-0.00022000000000000003, 0.272]      1   
2       0010067604  0.001014  (-0.00022000000000000003, 0.272]      1   
3       0010305739  0.001121  (-0.00022000000000000003, 0.272]      1   
4       0010102619  0.001134  (-0.00022000000000000003, 0.272]      1   
...            ...       ...                               ...    ...   
152505  0010187194  0.900752                    (0.715, 0.925]     20   
152506  0010223246  0.915061                    (0.715, 0.925]     20   
152507  0010067908  0.919779                    (0.715, 0.925]     20   
152508  0010069098  0.925294                    (0.715, 0.925]     20   
152509  0010211476  0.925294                    (0.715, 0.925]     20   

        SommeDeMONTANT_NUM  
0                     10.0  
1                     10.0  
2                     30.0  
3      

In [23]:
# INDIVIDUS SANS TARGET
test_target = vingtiles_ind_sans_target(model=model_17_cw, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/ind_sans_target_logistic_regression.xlsx")

ValueError: ignored

In [ ]:
####################################################################################
#                                FIN PREMIERS TESTS
####################################################################################

In [24]:
####################################################################################
#                                TEST AVEC LES ANCIENS PARAMETRES
####################################################################################
model_17_cw = LogisticRegression(random_state =0, C=0.01,penalty="l2" , solver="newton-cg", max_iter= 10000, class_weight="balanced")

evaluation(model_17_cw, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False )

test set : 
[[ 3486  2619]
 [ 8941 15456]]
              precision    recall  f1-score   support

           1       0.28      0.57      0.38      6105
           2       0.86      0.63      0.73     24397

    accuracy                           0.62     30502
   macro avg       0.57      0.60      0.55     30502
weighted avg       0.74      0.62      0.66     30502

train set : 
[[14175 10159]
 [35706 61968]]
              precision    recall  f1-score   support

           1       0.28      0.58      0.38     24334
           2       0.86      0.63      0.73     97674

    accuracy                           0.62    122008
   macro avg       0.57      0.61      0.56    122008
weighted avg       0.74      0.62      0.66    122008

GRAPHIQUE NON DEMANDE


In [25]:
# TRAIN SET TEST SET
vingtiles_tableau = vingtiles(model=model_17_cw, data_orig= data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_AAPOT21/Fichier_E_vingtiles_AA/train_test_logistic_regression.xlsx")

             IDENT     PROBA
107629  0010272515  0.081341
42884   0010109170  0.082445
144705  0010367114  0.083004
47633   0010121056  0.084642
108740  0010273739  0.086430
...            ...       ...
69768   0010176881  0.871663
29915   0010079712  0.871685
123802  0010312330  0.874939
86436   0010222278  0.876184
70521   0010177901  0.879319

[152510 rows x 2 columns]
0.843251    204
0.835865     76
0.792331     60
0.779294     49
0.838966     47
           ... 
0.541561      1
0.488397      1
0.790308      1
0.417559      1
0.536094      1
Name: PROBA, Length: 142316, dtype: int64
             IDENT     PROBA         TRANCHES CLASSE
107629  0010272515  0.081341  (0.0803, 0.283]      1
42884   0010109170  0.082445  (0.0803, 0.283]      1
144705  0010367114  0.083004  (0.0803, 0.283]      1
47633   0010121056  0.084642  (0.0803, 0.283]      1
108740  0010273739  0.086430  (0.0803, 0.283]      1
...            ...       ...              ...    ...
69768   0010176881  0.871663   (0.70

In [26]:
3633/7626

0.47639653815892996

In [27]:
3638/7625

0.4771147540983607

In [ ]:
####################################################################################################
#
#                                        DATA AUGMENTATION
#
####################################################################################################

In [ ]:
####################################################################################################
#
#                                       UNDER-SAMPLING  ET OVER-SAMPLING
#
####################################################################################################

In [ ]:
##########################################
# AVEC VARIABLES DONS ET ENCODAGE DUMMIES
##########################################

# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F",
                "Onoma-Phase_Scl", "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe Top_mon
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


In [ ]:

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

In [ ]:

# dimension du train set
print("Sans aucun traitement sur le train set : " , np.unique(train_Y_s, return_counts=True))

# diminution des données
t2 = TomekLinks(random_state=0)
X_resampled, y_resampled = t2.fit_resample(train_X_s[:,:nb_var] , train_Y_s )
print("Après under-sampling : ", np.unique(y_resampled, return_counts=True ))

# augmentation des données
tl = SMOTE(random_state=0)
X_resampled, y_resampled = tl.fit_resample(X_resampled, y_resampled )
print("Après over-smapling : " , np.unique(y_resampled, return_counts=True ))

'''
# enregistrement du nouveau train set car le traitement du under-sampling prendre du temps 
pd.DataFrame(np.concatenate((X_resampled, y_resampled.reshape((y_resampled.shape[0],1)) ),
                            axis=1)).to_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/sans_val_corr_spad_under_over_sampling.txt",
                                            sep=" ", index =False , header = False, encoding ="utf-8")
'''


Sans aucun traitement sur le train set :  (array([1, 2]), array([21397, 58039]))
Après under-sampling :  (array([1, 2]), array([21397, 52706]))
Après over-smapling :  (array([1, 2]), array([52706, 52706]))


'\n# enregistrement du nouveau train set car le traitement du under-sampling prendre du temps \npd.DataFrame(np.concatenate((X_resampled, y_resampled.reshape((y_resampled.shape[0],1)) ),\n                            axis=1)).to_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/sans_val_corr_spad_under_over_sampling.txt",\n                                            sep=" ", index =False , header = False, encoding ="utf-8")\n'

In [ ]:
'''
# Résultat du traitement sur le train set (under-sampling et over-sampling)
abc_test = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/sans_val_corr_spad_under_over_sampling.txt", delim_whitespace="\t", header = None, low_memory=False, dtype="Int64" ) #, dtype='Int64')
abc_np = abc_test.to_numpy(dtype="Int64")

X_resampled = abc_np[:,:-1]
y_resampled = abc_np[:, abc_np.shape[1] -1 ]
'''

In [ ]:
X_resampled.shape, y_resampled.shape

((105412, 269), (105412,))

In [ ]:
model_reg_logistic = LogisticRegression(random_state =0, max_iter= 10000, class_weight="balanced" )

param = {'C':[0.01,0.1,1], 'solver': ['liblinear','newton-cg', 'lbfgs'], 'penalty': ["l1","l2","none"] } 

grid_reg_logistic = GridSearchCV(model_reg_logistic, param_grid = param, cv=5, scoring ='recall', n_jobs=2 )
resultat_reg_logistic = grid_reg_logistic.fit(train_X_s, train_Y_s)
print(resultat_reg_logistic.best_params_)

{'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}


In [ ]:
model_17_cw = LogisticRegression(random_state =0,C=0.01, penalty="l2", solver="newton-cg", max_iter= 10000, class_weight="balanced")
evaluation(model_17_cw, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False )

test set : 
[[ 1051  4280]
 [ 1088 13441]]
              precision    recall  f1-score   support

           1       0.49      0.20      0.28      5331
           2       0.76      0.93      0.83     14529

    accuracy                           0.73     19860
   macro avg       0.62      0.56      0.56     19860
weighted avg       0.69      0.73      0.69     19860

train set : 
[[35256 17450]
 [ 3539 49167]]
              precision    recall  f1-score   support

           1       0.91      0.67      0.77     52706
           2       0.74      0.93      0.82     52706

    accuracy                           0.80    105412
   macro avg       0.82      0.80      0.80    105412
weighted avg       0.82      0.80      0.80    105412

GRAPHIQUE NON DEMANDE


In [ ]:
model_17_cw = LogisticRegression(random_state =0,C=1, penalty="l2", solver="newton-cg", max_iter= 100)
evaluation(model_17_cw, X_resampled, y_resampled, test_X_s, test_Y_s, scoring="recall", plot=False )

test set : 
[[  753  4578]
 [  695 13834]]
              precision    recall  f1-score   support

           1       0.52      0.14      0.22      5331
           2       0.75      0.95      0.84     14529

    accuracy                           0.73     19860
   macro avg       0.64      0.55      0.53     19860
weighted avg       0.69      0.73      0.67     19860

train set : 
[[34214 18492]
 [ 2098 50608]]
              precision    recall  f1-score   support

           1       0.94      0.65      0.77     52706
           2       0.73      0.96      0.83     52706

    accuracy                           0.80    105412
   macro avg       0.84      0.80      0.80    105412
weighted avg       0.84      0.80      0.80    105412

GRAPHIQUE NON DEMANDE


In [ ]:
#####################################################################################
###################################################################################
####################################################################################
####################################################################################

In [ ]:
############### A EFFACER